# Async Call Utilities - Concurrent Function Mapping with Retry and Control

The `ln` module provides powerful async utilities for applying functions to lists with comprehensive control:

**Core Functions:**
- **`alcall`**: Async list map with retry, concurrency control, and error handling
- **`bcall`**: Batch processing generator yielding results incrementally

**Key Features:**
- **Input/Output Processing**: Flatten, filter, deduplicate collections
- **Retry Logic**: Exponential backoff with configurable attempts
- **Timeout Control**: Per-call timeouts with cancellation
- **Concurrency Limits**: Semaphore-based max concurrent execution
- **Throttling**: Delay between task starts for rate limiting
- **Exception Handling**: Return or raise exceptions with proper grouping
- **Order Preservation**: Results maintain input order regardless of completion time

In [1]:
import asyncio

from lionherd_core.ln import AlcallParams, alcall, bcall

## 1. Basic Usage - Async Mapping

`alcall` applies a function to each list element concurrently, handling both sync and async functions.

In [2]:
# Sync function
def double(x):
    return x * 2


# Async function
async def async_double(x):
    await asyncio.sleep(0.01)  # Simulate async work
    return x * 2


# Apply sync function
result_sync = await alcall([1, 2, 3, 4], double)
print(f"Sync function: {result_sync}")

# Apply async function
result_async = await alcall([1, 2, 3, 4], async_double)
print(f"Async function: {result_async}")

Sync function: [2, 4, 6, 8]
Async function: [2, 4, 6, 8]


## 2. Input Processing - Clean and Transform Input

Process input before applying the function: flatten nested structures, remove None values, deduplicate.

**Note:** When flattening without `dropna`, None values are included and may cause errors unless handled with `return_exceptions=True`.

In [3]:
# Nested list with duplicates and None
nested_input = [[1, 2], [3, None], [1, 4], None, 5]

# Flatten only - Note: None values cause errors unless handled
result_flatten = await alcall(nested_input, double, input_flatten=True, return_exceptions=True)
print(f"Flatten: {result_flatten}")

# Flatten + drop None - this removes None values before processing
result_dropna = await alcall(nested_input, double, input_flatten=True, input_dropna=True)
print(f"Flatten + dropna: {result_dropna}")

# Flatten + drop None + unique
result_unique = await alcall(
    nested_input, double, input_flatten=True, input_dropna=True, input_unique=True
)
print(f"Flatten + dropna + unique: {result_unique}")

Flatten: [2, 4, 6, TypeError("unsupported operand type(s) for *: 'NoneType' and 'int'"), 2, 8, TypeError("unsupported operand type(s) for *: 'NoneType' and 'int'"), 10]
Flatten + dropna: [2, 4, 6, 2, 8, 10]
Flatten + dropna + unique: [2, 4, 6, 8, 10]


## 3. Output Processing - Transform Results

Same processing options available for output: flatten nested results, filter None, deduplicate.

In [4]:
# Function that returns nested lists
def make_nested(x):
    if x == 2:
        return None  # Some items return None
    return [[x, x], [x * 10]]


# Raw output
result_raw = await alcall([1, 2, 3], make_nested)
print(f"Raw: {result_raw}")

# Flatten output
result_flatten_out = await alcall([1, 2, 3], make_nested, output_flatten=True)
print(f"Flatten output: {result_flatten_out}")

# Flatten + drop None
result_clean = await alcall([1, 2, 3], make_nested, output_flatten=True, output_dropna=True)
print(f"Flatten + dropna output: {result_clean}")

Raw: [[[1, 1], [10]], None, [[3, 3], [30]]]
Flatten output: [1, 1, 10, None, 3, 3, 30]
Flatten + dropna output: [1, 1, 10, 3, 3, 30]


## 4. Retry Mechanism - Handle Transient Failures

Configure retry attempts with exponential backoff for resilient operations.

In [5]:
# Flaky function that fails first 2 attempts
attempt_count = {}


def flaky_func(x):
    attempt_count[x] = attempt_count.get(x, 0) + 1
    if attempt_count[x] < 3:  # Fail first 2 attempts
        raise ValueError(f"Attempt {attempt_count[x]} failed for {x}")
    return x * 2


# Reset counter
attempt_count.clear()

# Retry up to 3 times with 0.01s initial delay, 2x backoff
result = await alcall(
    [1, 2, 3],
    flaky_func,
    retry_attempts=3,
    retry_initial_delay=0.01,
    retry_backoff=2.0,
)
print(f"Result: {result}")
print(f"Attempts per item: {attempt_count}")

Result: [2, 4, 6]
Attempts per item: {1: 3, 2: 3, 3: 3}


In [6]:
# Use retry_default to return value instead of raising on exhaustion
def always_fails(x):
    raise ValueError(f"Always fails for {x}")


result_default = await alcall(
    [1, 2, 3],
    always_fails,
    retry_attempts=2,
    retry_default=-1,  # Return -1 instead of raising
)
print(f"With retry_default: {result_default}")

With retry_default: [-1, -1, -1]


## 5. Timeout Control - Limit Execution Time

Set per-call timeouts to prevent hanging operations.

In [7]:
async def slow_func(x):
    if x == 2:
        await asyncio.sleep(1.0)  # This will timeout
    else:
        await asyncio.sleep(0.01)
    return x * 2


# Timeout after 0.1 seconds, return -1 on timeout
result_timeout = await alcall(
    [1, 2, 3],
    slow_func,
    retry_timeout=0.1,
    retry_default=-1,  # TimeoutError → -1
)
print(f"With timeout (item 2 timed out): {result_timeout}")

With timeout (item 2 timed out): [2, -1, 6]


## 6. Concurrency Control - Limit Parallel Execution

Use `max_concurrent` to limit simultaneous executions (e.g., API rate limits, resource constraints).

In [8]:
# Track concurrent executions
concurrent_count = 0
max_seen = 0


async def track_concurrent(x):
    global concurrent_count, max_seen
    concurrent_count += 1
    max_seen = max(max_seen, concurrent_count)
    await asyncio.sleep(0.05)
    concurrent_count -= 1
    return x


# Reset counters
concurrent_count = 0
max_seen = 0

# Limit to 2 concurrent executions
result_limited = await alcall([1, 2, 3, 4, 5], track_concurrent, max_concurrent=2)
print(f"Result: {result_limited}")
print(f"Max concurrent executions: {max_seen} (limit was 2)")

Result: [1, 2, 3, 4, 5]
Max concurrent executions: 2 (limit was 2)


## 7. Throttling - Rate Limit Task Starts

Add delay between starting tasks to control burst rate (different from max_concurrent).

In [9]:
import time

start_times = []


async def record_start(x):
    start_times.append(time.time())
    return x


# Reset
start_times.clear()

# 0.05s delay between starting each task
result_throttle = await alcall([1, 2, 3, 4], record_start, throttle_period=0.05)

# Calculate intervals
intervals = [start_times[i] - start_times[i - 1] for i in range(1, len(start_times))]
print(f"Start time intervals: {[f'{x:.3f}s' for x in intervals]}")
print(f"All intervals >= 0.05s: {all(x >= 0.04 for x in intervals)}")

Start time intervals: ['0.051s', '0.050s', '0.051s']
All intervals >= 0.05s: True


## 8. Exception Handling - Control Error Behavior

Choose to return exceptions as values or raise them as ExceptionGroup.

In [10]:
def selective_fail(x):
    if x == 2:
        raise ValueError(f"Failed on {x}")
    return x * 2


# Return exceptions as values
result_exceptions = await alcall([1, 2, 3], selective_fail, return_exceptions=True)
print(f"Results (with exception): {result_exceptions}")
print(f"Exception at index 1: {isinstance(result_exceptions[1], ValueError)}")
print(f"Exception message: {result_exceptions[1]}")

Results (with exception): [2, ValueError('Failed on 2'), 6]
Exception at index 1: True
Exception message: Failed on 2


In [11]:
# Raise exceptions (default behavior)
try:
    result_raise = await alcall([1, 2, 3], selective_fail, return_exceptions=False)
except ExceptionGroup as eg:
    print(f"Caught ExceptionGroup with {len(eg.exceptions)} exception(s)")
    print(f"First exception: {eg.exceptions[0]}")

Caught ExceptionGroup with 1 exception(s)
First exception: Failed on 2


## 9. Order Preservation - Results Match Input Order

Results are returned in input order regardless of completion time.

In [12]:
async def variable_delay(x):
    # Later items finish first (reverse order completion)
    await asyncio.sleep(0.05 * (5 - x))
    return x * 10


result_ordered = await alcall([1, 2, 3, 4], variable_delay)
print("Input order: [1, 2, 3, 4]")
print(f"Result (preserved order): {result_ordered}")
print(f"Matches expected [10, 20, 30, 40]: {result_ordered == [10, 20, 30, 40]}")

Input order: [1, 2, 3, 4]
Result (preserved order): [10, 20, 30, 40]
Matches expected [10, 20, 30, 40]: True


## 10. Batch Processing - bcall for Incremental Results

`bcall` processes input in batches, yielding results incrementally (useful for large datasets).

In [13]:
async def process_item(x):
    await asyncio.sleep(0.01)
    return x * 2


# Process 10 items in batches of 3
all_results = []
batch_num = 0

async for batch_result in bcall(list(range(1, 11)), process_item, batch_size=3):
    batch_num += 1
    print(f"Batch {batch_num}: {batch_result}")
    all_results.extend(batch_result)

print(f"\nAll results: {all_results}")
print(f"Total items: {len(all_results)}")

Batch 1: [2, 4, 6]
Batch 2: [8, 10, 12]
Batch 3: [14, 16, 18]
Batch 4: [20]

All results: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
Total items: 10


## 11. Parameter Objects - Reusable Configurations

Use `AlcallParams` to define reusable parameter sets.

In [14]:
# Create reusable parameter configuration
params = AlcallParams(
    retry_attempts=2,
    retry_initial_delay=0.01,
    retry_backoff=2.0,
    max_concurrent=3,
)

# Use params with different inputs/functions
# Additional parameters like return_exceptions can be passed as keyword arguments
result1 = await params([1, 2, 3], double, return_exceptions=True)
result2 = await params([4, 5, 6], async_double, return_exceptions=True)

print(f"Result 1: {result1}")
print(f"Result 2: {result2}")

Result 1: [2, 4, 6]
Result 2: [8, 10, 12]


## 12. Complex Example - Real-World API Simulation

Combine multiple features for robust API-like operations.

In [15]:
# Simulate API with rate limits and transient failures
api_call_count = 0
failure_ids = {3, 5}  # These IDs fail first attempt
attempt_tracker = {}


async def api_fetch(user_id):
    global api_call_count
    api_call_count += 1
    attempt_tracker[user_id] = attempt_tracker.get(user_id, 0) + 1

    # Simulate transient failures
    if user_id in failure_ids and attempt_tracker[user_id] == 1:
        raise ConnectionError(f"Network error for user {user_id}")

    # Simulate API delay
    await asyncio.sleep(0.02)
    return {"user_id": user_id, "name": f"User{user_id}", "score": user_id * 10}


# Reset tracking
api_call_count = 0
attempt_tracker.clear()

# Fetch user data with:
# - Max 3 concurrent requests (rate limit)
# - Retry transient failures (up to 2 retries)
# - 0.5s timeout per request
# - Return exceptions instead of crashing
user_ids = [1, 2, 3, 4, 5, 6]
results = await alcall(
    user_ids,
    api_fetch,
    max_concurrent=3,
    retry_attempts=2,
    retry_initial_delay=0.01,
    retry_timeout=0.5,
    return_exceptions=True,
)

print(f"Total API calls: {api_call_count}")
print(f"Attempts per user: {attempt_tracker}")
print("\nResults:")
for i, result in enumerate(results):
    if isinstance(result, Exception):
        print(f"  User {user_ids[i]}: ERROR - {result}")
    else:
        print(f"  User {user_ids[i]}: {result}")

Total API calls: 8
Attempts per user: {1: 1, 2: 1, 3: 2, 4: 1, 5: 2, 6: 1}

Results:
  User 1: {'user_id': 1, 'name': 'User1', 'score': 10}
  User 2: {'user_id': 2, 'name': 'User2', 'score': 20}
  User 3: {'user_id': 3, 'name': 'User3', 'score': 30}
  User 4: {'user_id': 4, 'name': 'User4', 'score': 40}
  User 5: {'user_id': 5, 'name': 'User5', 'score': 50}
  User 6: {'user_id': 6, 'name': 'User6', 'score': 60}


## Summary Checklist

**Async Call Utilities Essentials:**
- ✅ Apply functions to lists concurrently with `alcall`
- ✅ Process batches incrementally with `bcall` generator
- ✅ Clean input: flatten, dropna, unique
- ✅ Transform output: same processing options
- ✅ Retry failed operations with exponential backoff
- ✅ Set per-call timeouts to prevent hanging
- ✅ Limit concurrent execution with semaphores
- ✅ Throttle task starts for rate limiting
- ✅ Handle exceptions: return as values or raise as group
- ✅ Preserve input order regardless of completion time
- ✅ Support both sync and async functions transparently
- ✅ Reuse configurations with `AlcallParams`

**Next Steps:**
- See `to_list` for input/output processing details
- See `concurrency` module for underlying primitives
- Use in production for robust API calls, data processing, and parallel workflows